In [1]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout
from keras import backend as K
import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import MaxPool2D
import tensorflow as tf

In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model

from keras.layers import Input, Lambda, Dense, Flatten, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import applications

In [3]:
train_path="E:/SBIR/Selected/train"
test_path="E:/SBIR/Selected/test"
class_names=os.listdir(train_path)
class_names_test=os.listdir(test_path)

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                            shear_range = 0.4,
                            zoom_range = 0.4,
                            horizontal_flip = True,
                            vertical_flip = True,
                            validation_split = 0.2)
test_datagen = ImageDataGenerator()

In [6]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=(224, 224),batch_size=64,shuffle=True,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=64,shuffle=False,class_mode='categorical')

Found 780 images belonging to 13 classes.
Found 260 images belonging to 13 classes.


In [7]:
incep_base = applications.InceptionV3(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
incep_base.trainable = False

inputs = Input(shape=(224, 224, 3))

x = incep_base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(13, activation = 'softmax')(x)
incep_model = Model(inputs, outputs)

In [8]:
incep_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 13)                13325     
                                                             

In [9]:
incep_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss= keras.losses.CategoricalCrossentropy(from_logits = True),
    metrics= [keras.metrics.CategoricalAccuracy()],
)

In [10]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Define ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001)

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
es = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
H = incep_model.fit(train_generator,
                    validation_data=test_generator,
                    epochs=25,
                    callbacks=[es,reduce_lr])

Epoch 1/25
13/13 [==============================] - 30s 2s/step - loss: 0.7604 - categorical_accuracy: 0.7756 - val_loss: 56.9738 - val_categorical_accuracy: 0.0962 - lr: 1.0000e-04
Epoch 2/25
13/13 [==============================] - 29s 2s/step - loss: 0.6909 - categorical_accuracy: 0.7949 - val_loss: 57.8231 - val_categorical_accuracy: 0.0769 - lr: 1.0000e-04
Epoch 3/25
13/13 [==============================] - 29s 2s/step - loss: 0.7067 - categorical_accuracy: 0.7782 - val_loss: 63.7284 - val_categorical_accuracy: 0.0731 - lr: 1.0000e-04
Epoch 4/25
13/13 [==============================] - 28s 2s/step - loss: 0.6203 - categorical_accuracy: 0.8103 - val_loss: 61.6490 - val_categorical_accuracy: 0.0654 - lr: 1.0000e-04
Epoch 5/25
 4/13 [========>.....................] - ETA: 17s - loss: 0.6033 - categorical_accuracy: 0.8008

KeyboardInterrupt: 